In [ ]:
!pip install langchain-groq langchain-community huggingface_hub PyPDF2 langchain_huggingface faiss-cpu dotenv gradio

from langchain_groq import ChatGroq
import gradio as gr
from PyPDF2 import PdfReader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from huggingface_hub import login
from dotenv import load_dotenv
import os
import io

load_dotenv()
login(os.getenv("hugging face key"))

# -------------------------------
# Extract Text from PDF
def get_pdf_text(pdf_file):
    reader = PdfReader(pdf_file)
    raw_text = ""
    for page in reader.pages:
        if page.extract_text():
            raw_text += page.extract_text()
    return raw_text

# -------------------------------
# Text Chunking
def get_text_chunks(raw_text):
    splitter = CharacterTextSplitter(
        separator="\n", chunk_size=1000, chunk_overlap=200, length_function=len
    )
    return splitter.split_text(raw_text)

# -------------------------------
# Vector Store with HuggingFace Embeddings
def get_vectorstore(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_texts(chunks, embedding=embeddings)
    return vectorstore

# -------------------------------
# LLM Setup
from google.colab import userdata

llm = ChatGroq(
    model_name="llama3-70b-8192",
    temperature=0.7,
    request_timeout=30,
    api_key="grok Key"
)

# -------------------------------
# QA RAG Chain
def create_qa_chain(vectorstore):
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    prompt_template = """
    You are a helpful assistant trained on product manuals.
    Answer the user's question based on the context provided below.

    
    {context}
    

    Question: {question}
    Answer:
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain

# -------------------------------
# QA Function for Gradio
def product_manual_qa(pdf_file, user_question):
    if not pdf_file or not user_question:
        return "⚠ Please upload a PDF and enter a question."

    raw_text = get_pdf_text(pdf_file)
    chunks = get_text_chunks(raw_text)
    vectorstore = get_vectorstore(chunks)
    qa_chain = create_qa_chain(vectorstore)

    return qa_chain.invoke(user_question)

# -------------------------------
# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 📘 Product Manual Question Answering System")

    with gr.Row():
        pdf_input = gr.File(label="📄 Upload Product Manual PDF", file_types=[".pdf"])
        user_question = gr.Textbox(label="❓ Ask a Question about the Manual")

    submit_button = gr.Button("🔍 Get Answer")
    answer_output = gr.Textbox(label="📤 Answer", lines=10)

    submit_button.click(
        fn=product_manual_qa,
        inputs=[pdf_input, user_question],
        outputs=answer_output
    )

demo.launch()